In [1]:
import os
import numpy as np
import pretty_midi
from sklearn.model_selection import train_test_split
import tensorflow as tf

MIDI_DIR = "data/archive/"


def midi_to_notes(midi_path):
    midi = pretty_midi.PrettyMIDI(midi_path)

    notes = []
    for instrument in midi.instruments:
        for note in instrument.notes:
            notes.append([
                note.pitch
            ])
    if len(notes) == 0:
        return None
    return np.array(notes)

all_notes = []
for file in os.listdir(MIDI_DIR):
    if file.endswith(".mid") or file.endswith(".midi"):
        note_data = midi_to_notes(os.path.join(MIDI_DIR, file))
        if note_data is not None:
            all_notes.append(note_data)

all_notes = np.concatenate(all_notes, axis=0)
print("Shape of all notes:", all_notes.shape)

note_tensor = tf.convert_to_tensor(all_notes, dtype=tf.int32)
SEQ_LEN = 10

def make_sequences(note_tensor, seq_len=SEQ_LEN):
    X, y = [], []
    notes = note_tensor.numpy()
    for i in range(len(notes) - seq_len):
        X.append(notes[i:i+seq_len])
        y.append(notes[i+seq_len])
    return tf.convert_to_tensor(X, dtype=tf.int32), tf.convert_to_tensor(y, dtype=tf.int32)

X_train, y_train = make_sequences(note_tensor)
X_test, y_test = make_sequences(note_tensor)

X_train = tf.squeeze(X_train, axis=-1)
X_test = tf.squeeze(X_test, axis=-1)


2025-11-02 10:32:54.047862: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/user/pythonenv/default/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:122: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Shape of all notes: (4060, 1)


I0000 00:00:1762075977.530908  209174 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13321 MB memory:  -> device: 0, name: AMD Radeon RX 9070 XT, pci bus id: 0000:03:00.0


In [2]:
from tensorflow.keras import layers, models

vocab_size = 128
embed_dim = 64
lstm_units = 128
num_classes = 128


model = models.Sequential([
    layers.Input(shape=(None,)),
    layers.Embedding(input_dim=vocab_size, output_dim=embed_dim),
    layers.LSTM(lstm_units, return_sequences=True),
    layers.LSTM(lstm_units),
    layers.Dense(num_classes, activation='softmax')
])


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 64)       │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 255,104 (996.50 KB)

 Trainable params: 255,104 (996.50 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping



early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=1000,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1,
)
val_loss, val_acc = model.evaluate(X_test, y_test)
print(f"\n✅ Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_acc:.4f}")

Epoch 1/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.0291 - loss: 4.1613 - val_accuracy: 0.0363 - val_loss: 3.8687
Epoch 2/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0346 - loss: 3.8786 - val_accuracy: 0.0378 - val_loss: 3.8630
Epoch 3/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.0336 - loss: 3.8680 - val_accuracy: 0.0338 - val_loss: 3.8524
Epoch 4/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0328 - loss: 3.8678 - val_accuracy: 0.0358 - val_loss: 3.8522
Epoch 5/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0319 - loss: 3.8640 - val_accuracy: 0.0358 - val_loss: 3.8515
Epoch 6/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.0343 - loss: 3.8583 - val_accuracy: 0.0343 - val_loss: 3.8428
Epoch 7/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0353 - loss: 3.8586 - val_accuracy: 0.0319 - val_loss: 3.8461
Epoch 8/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0336 - loss: 3.8534 - val_accu

In [10]:

def sample_from_probs(preds, temperature):

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_music(model, seed_sequence, num_steps=20, temperature=4.0):

    generated = []
    current_seq = seed_sequence

    for _ in range(num_steps):
        input_seq = tf.expand_dims(current_seq, axis=0)
        preds = model.predict(input_seq, verbose=0)[0]

        next_pitch = sample_from_probs(preds, temperature)
        generated.append(next_pitch)

        current_seq = np.append(current_seq[1:], next_pitch)

    return np.array(generated)

seed_seq = X_train[0]
generated_notes = generate_music(model, seed_seq, num_steps=20)
print("Generated notes shape:", generated_notes.shape)


def pitches_to_midi(pitch_array, output_file, start_time=0.0, step=0.5, velocity=100):

    midi = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=4)

    time = start_time
    for pitch in pitch_array:
        pitch = int(np.clip(pitch, 0, 127))
        piano.notes.append(pretty_midi.Note(
            velocity=velocity,
            pitch=pitch,
            start=time,
            end=time + step
        ))
        time += step

    midi.instruments.append(piano)
    midi.write(output_file)
    print(f"MIDI saved to {output_file}")


pitches_to_midi(generated_notes, "generated_music3.mid")

Generated notes shape: (20,)
MIDI saved to generated_music3.mid
